# Estimating a model with a known number of events

This section will show you how to estimate a single HMP model with a given number of event as in the previous but we'll make it harder to find.

In [ ]:
## Importing the package
import hsmm_mvpy as hmp
#Plotting 
import numpy as np
import matplotlib.pyplot as plt

cpus = 3 # For multiprocessing, usually a good idea to use multiple CPus as long as you have enough RAM

## Running the demo function in the simulation module
# Inspect the given function to apply your own simulations
from hsmm_mvpy.simulations import demo
number_of_events = 8 #how many events to simulate
epoch_data, random_source_times, info = demo(cpus, number_of_events)

Then we transform the data:
- Standardize individual variances (not in this case as only one participant is simulated)
- Reducing dimensionality by selecting a number of principal component from the spatial PCA applied to the variance-covariance matrix of the channel activity
- zscoring the data of each participant

In [ ]:
hmp_dat = hmp.utils.transform_data(epoch_data, apply_standard=False, n_comp=4)
print(hmp_dat)

HMP uses classes to instantiate fit of hmp models. Therefore before estimating a model one needs to declare an instance of the hmp class by specifying the data, the sampling frequency of the data as well as several parmeters such as the event width (in milliseconds) and how many CPU to use.

In [ ]:
init = hmp.models.hmp(hmp_dat, sfreq=epoch_data.sfreq, event_width=50, cpus=cpus, location=25)#Initialization of the model

Once the class has been initiated, the function ```fit_single()``` can be used to estimate one hmp model with a specified number of transition events

## Direct estimation
We can simply call the ```fit_single``` method from the hmp class to estimate a given number of events. The hmp algorithm will then look for 4 events. Now the expectation maximization algorithm to fit the model is in this case sensitive to the starting points given to the stage duration. By default, the ```fit_single``` method uses a starting point where all events are equally distributed between 0 and the mean RT.

In [ ]:
#Fitting
selected = init.fit_single(number_of_events)#function to fit an instance of a 10 events model

#Visualizing
hmp.visu.plot_topo_timecourse(epoch_data, selected, info, init, magnify=1, sensors=False,
                                times_to_display = np.mean(np.cumsum(random_source_times,axis=1),axis=0))

And we can take a look at the traces from the expectation maximization procedure showing us how many iterations it needed and at what Log-likelihood (LL) it stopped according to the default tolerance on the increase in LL (default 1e-4)

In [ ]:
plt.plot(selected.traces)
plt.ylabel('Log-likelihood')
plt.xlabel('EM iteration');


Now launching a single model without additional starting points is not a good idea as often the default starting values (illustrated below) of the expectation maximization algorithm will end up in a local minima: 
 

In [ ]:
#Fitting
selected = init.fit_single(number_of_events, maximization=False)#function to fit an instance of a 10 events model

#Visualizing
hmp.visu.plot_topo_timecourse(epoch_data, selected, info, init, magnify=1, sensors=False,
                                times_to_display = np.mean(np.cumsum(random_source_times,axis=1),axis=0))

We need to use more starting points!

### Exercise 1:

Adapt the cell below to generate starting points on the scale of the gamma parameters to capture all those events.

Keep in mind that the mean of a gamma is _scale * shape_ 

In [ ]:
pars = np.reshape(np.concatenate([
    np.repeat(init.shape, 9), 
    [1,1,1,1,1,1,1,1,1]]),#Replace values here
                       [2,9]).T
pars

In [ ]:
#Fitting
selected = init.fit_single(number_of_events, parameters=pars)#function to fit an instance of a 10 events model

#Visualizing
hmp.visu.plot_topo_timecourse(epoch_data, selected, info, init, magnify=1, sensors=False,
                                times_to_display = np.mean(np.cumsum(random_source_times,axis=1),axis=0))

## Random method


A better idea is to run a single model with several starting points and selecting the result with the best fit. This can be declared in the ```single_fit()``` function; hereby an example with 100 random starting points.

In [ ]:
#NOTE: if you run this in VS code on a Mac in an interactive window, you might get strange behavior (running of previous processes) due to the multiprocessing. If so, set your multiprocessing start method to 'fork' (which is the default on Unix) by uncommenting the next two lines:
#import multiprocessing as mp
#mp.set_start_method("fork")

# Fitting
selected = init.fit_single(number_of_events, method='random', starting_points=100)#function to fit an instance of a 4 events model
hmp.visu.plot_topo_timecourse(epoch_data, selected, info, init, magnify=1, sensors=False,
                                times_to_display = np.mean(np.cumsum(random_source_times,axis=1),axis=0))

But, by definition, the starting points are random so inducing a lot of redundancy and also taking the risk that some points in the parameter space remain unexplored. Several calls to this function will not always give the correct solution.

### Exercise 2:

What good solution for this can you think of? To illustrate where we are going, using default starting points fit models by subsampling from 1 to 12 events (e.g. 2, 4, 10, 12)

In [ ]:
selected = init.fit_single(number_of_events)#Replace number_of_events with the desired number
hmp.visu.plot_topo_timecourse(epoch_data, selected, info, init, magnify=1, sensors=False,
                                times_to_display = np.mean(np.cumsum(random_source_times,axis=1),axis=0))

## Backward estimation

Another solution than using random points is to first estimate a model with the maximal number of possible events (referred to as 'the maximal model'), and then decrease the number of events one by one.

The idea is that genuine events will necessarily be found at the expected locations in the maximal model. Because the backward estimation method iteratively removes the weakest event (in terms of likelihood), only the 'strongest' events remain even if there location would have been hard to find with a single fit and default starting values.

In order to do this we will use the ```backward_estimation()``` function. This function first estimates the maximal model (defined based on the event width and the minimum reaction time), then estimates the max_event - 1 solution by iteratively removing one of the events and picking the next solution with the highest likelihood (for more details see Borst & Anderson, [2021](http://jelmerborst.nl/pubs/ACTR_hmp_MVPA_BorstAnderson_preprint.pdf)) and repeat this until the 1 event solution is reached. 

In [ ]:
%%time 
bests = init.backward_estimation(max_events=int(init.compute_max_events()/2)) #/2 to go faster

Here we plot the resulting solutions going from the maximal possible number of events that fit into the minimum RT given a minimum duration of 25ms in this example. 

In [ ]:
hmp.visu.plot_topo_timecourse(epoch_data, bests, info, init, sensors=False,
                    times_to_display = np.mean(np.cumsum(random_source_times,axis=1),axis=0), magnify=1)

And from these solutions we can then select the number of events we originally wanted to estimate (which is the correct solution in this case):

In [ ]:
selected = bests.sel(n_events=number_of_events)
hmp.visu.plot_topo_timecourse(epoch_data, selected, info, init, sensors=False,
                                times_to_display = np.mean(np.cumsum(random_source_times,axis=1),axis=0), magnify=1)

The downside is that we are still unsure about whether we included all possible starting points in the mix. In addition, this method can be suboptimal with 1) long RTs and therefore a lot of events to fit and long computation times and 2) if there is a big difference between the minimum RT (determining the maximum number of events) and the mean RT (all possible locations of these events).

# Sliding events

To account for the second point in the previous comment, one possibility is to estimate a single event model and test the landing point in the log-likelihood. The sliding event works by sliding a single event separating the mean reaction time into two stages with the first one ranging from 0 to the mean reaction time. Discontinuities in the resulting log-likelihood usually shows that there is an event around the corresponding times. Note though that it is not a perfect process and a very high variance event can cover a near lower variance event (e.g. between sample number 180 and 250)

In [ ]:
init.sliding_event(fix_pars=True)

Now the previous curve is obtaine by fixing the parameters of the two gammas and only estimating channel contribution but if we relax this constrain:

In [ ]:
init.sliding_event(fix_pars=False, step=3)#step 3 test every 3rd sample 

Then we see that each sample number used as starting point will go to its nearest local minima. But based on the number of landing points we see, we did not capture all the events that we simulated (8). The reason is that, while we did explore the parameter space of the gammas, we didn't test for different starting points in the channel contributions. If we generate a lot of magnitudes and pass it to the ```sliding_event``` function the landing points look closer to what we expect.

In [ ]:
magnitudes_starting_points = init.gen_mags(n_events=1, decimate=3)#decimate reduces the possible number of starting points
magnitudes_starting_points[::2]

In [ ]:
init.sliding_event(magnitudes=magnitudes_starting_points, step=3)

It appears that several landings points appear with some redundancies. To group those points together we use a clustering algorithm and make a choice on the number of clusters based on the grouping quality (Silhouette and visual inspection).

In [ ]:
#calc estimates
lkhs, mags, channels, times = init.sliding_event_mags(epoch_data, step=3, decimate_grid = 3, cpu=cpus, plot=True, min_iteration=10)

# cluster in time, lkh, and mags
mags_cl, pars_cl = hmp.clusters.cluster_events(init, lkhs, mags, channels, times, method='time_x_lkh_x_mags', max_clust=10, p_outlier=.05, info=info, calc_outliers=True)

#fit final model
best_estimate = init.fit_single(mags_cl.shape[0], magnitudes=mags_cl,parameters=pars_cl)
hmp.visu.plot_topo_timecourse(epoch_data, best_estimate, info, init, magnify=1, sensors=False, time_step=1000/init.sfreq,xlabel='Time (ms)', contours=0, event_lines=True, colorbar=True)

In [ ]:
hmp.visu.plot_topo_timecourse(epoch_data, best_estimate, info, init, magnify=1, sensors=False, time_step=1000/init.sfreq, xlabel='Time (ms)', contours=0, event_lines=True, colorbar=True,
                              times_to_display = np.mean(np.cumsum(random_source_times,axis=1),axis=0))

Now the solution isn't yet perfect and can take quite some time depending on the number of Principal components (and therefore combinations of starting points) and the number of tested clusters. One possible explanation is that not taking into account the previous events makes it harder to detect the next one (probably why we are missing the last one here). Therefore we created a method combining all the properties of this way of exploring the starting points:

## Cumulative event fit

Instead of fitting an _n_ event model, as the ```sliding_event()``` function, this method starts by fitting a 1 event model (two stages) using each sample from the time 0 (stimulus onset) to the mean RT. Therefore it tests for the landing point of the expectation maximization algorithm given each sample as starting point and the likelihood associated with this landing point. As soon as a starting points reaches the convergence criterion, the function fits an _n+1_ event model and uses the next samples in the RT for the following event.

In [ ]:
estimates = init.fit()

In [ ]:
hmp.visu.plot_topo_timecourse(epoch_data, estimates, info, init, 
                                times_to_display = np.mean(np.cumsum(random_source_times,axis=1),axis=0), magnify=1)

### Exercise 3:

What can cause the failure to capture the three successive events knowing that their topologies are quite similar

In [ ]:
selected = bests.sel(n_events=number_of_events)
hmp.visu.plot_topo_timecourse(epoch_data, selected, info, init, sensors=False,
                                times_to_display = np.mean(np.cumsum(random_source_times,axis=1),axis=0), magnify=1)

In [ ]:
hmp_dat = hmp.utils.transform_data(epoch_data, apply_standard=False)
#Plot new PC selection
hmp.visu.plot_components_sensor(hmp_dat, positions=info)
#Re-Initialization of the model
init = hmp.models.hmp(hmp_dat, sfreq=epoch_data.sfreq, event_width=50, cpus=cpus, location=25)

Reperform the fit

In [ ]:
estimates = init.fit()

In [ ]:
hmp.visu.plot_topo_timecourse(epoch_data, estimates, info, init, 
                                times_to_display = np.mean(np.cumsum(random_source_times,axis=1),axis=0), magnify=1)

# Data saving

Once finished we can save fitted models using the dedicated command

In [ ]:
hmp.utils.save_fit(selected, 'selected.nc')

And load the same way

In [ ]:
estimates = hmp.utils.load_fit('selected.nc')

Or even only save the estimated event probabilities 

In [ ]:
hmp.utils.save_eventprobs(selected.eventprobs, 'selected_eventprobs.nc')